In [7]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os
import numpy as np

In [30]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [31]:
os.listdir('MNIST_data')

['t10k-labels-idx1-ubyte.gz',
 'train-labels-idx1-ubyte.gz',
 'train-images-idx3-ubyte.gz',
 't10k-images-idx3-ubyte.gz']

In [83]:
x = tf.placeholder(tf.float32, [None, 784], name='x')
variable_summaries(x)
W = tf.Variable(tf.zeros( [784, 10]), name='W')
variable_summaries(W)
b  = tf.Variable(tf.zeros([10]), name='b')
variable_summaries(b)
y = tf.nn.softmax(tf.matmul(x, W) + b, name='y')
y_ = tf.placeholder(tf.float32, [None, 10], name='y_')

In [48]:
cross_entropy = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y), name='cross_entropy')
tf.summary.scalar('cross_entropy', cross_entropy)

<tf.Tensor 'cross_entropy:0' shape=() dtype=string>

In [49]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [50]:
sess = tf.InteractiveSession()

In [51]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1), name='correct_prediction')

In [52]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')
tf.summary.scalar('accuracy', accuracy)

<tf.Tensor 'accuracy:0' shape=() dtype=string>

In [55]:
merged = tf.summary.merge_all()
my_writer = tf.summary.FileWriter('/home/jupyter/tlog/oldgraph/',sess.graph)
tf.global_variables_initializer().run()

In [98]:
# for i in range(1000):
#     if i%10 == 0:        
#         batch_xs, batch_ys = mnist.train.next_batch(100)
#         summary, acc = sess.run([merged, accuracy], feed_dict={x: mnist.test.images, y_: mnist.test.labels})
#         my_writer.add_summary(summary, i)
# #         print('Accuracy at step %s: %s' % (i, acc))
#     else:
#         summary, _ = sess.run([merged, train_step], feed_dict={x: batch_xs, y_: batch_ys})
#         my_writer.add_summary(summary)

# CNN Implementation

In [100]:
tf.reset_default_graph()

In [101]:
def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)

In [102]:
x = tf.placeholder(tf.float32, [None, 784], name='x')
variable_summaries(x)

In [112]:
W = tf.Variable(tf.zeros( [784, 10]), name='W')
variable_summaries(W)
b  = tf.Variable(tf.zeros([10]), name='b')
variable_summaries(b)
y = tf.nn.softmax(tf.matmul(x, W) + b, name='y')
y_ = tf.placeholder(tf.float32, [None, 10], name='y_')

In [103]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name='weights')

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name='bias')

In [104]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME', name='conv2d')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME', name='mp22')

In [105]:
with tf.name_scope('layer1') as scope:
    W_conv1 = weight_variable([5, 5, 1, 32])
    b_conv1 = bias_variable([32])

    x_image = tf.reshape(x, [-1, 28, 28, 1])

    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)

In [106]:
with tf.name_scope('layer2') as scope:
    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])

    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)

In [107]:
with tf.name_scope('denselayer') as scope:
    W_fc1 = weight_variable([7 * 7 * 64, 1024])
    b_fc1 = bias_variable([1024])

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [108]:
with tf.name_scope('dropout') as scope:
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [109]:
with tf.name_scope('readout') as scope:
    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])

    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [113]:
with tf.name_scope('evaluation') as scope:
    cross_entropy = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv), name='cross_entropy')
    tf.summary.scalar('cross_entropy', cross_entropy)
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1), name='correct_prediction')
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')
    tf.summary.scalar('accuracy', accuracy)

In [115]:
with tf.Session() as sess:
    merged = tf.summary.merge_all()
    my_writer = tf.summary.FileWriter('/home/jupyter/tlog/newcnn/',sess.graph)
    sess.run(tf.global_variables_initializer())
    for i in range(2000):
        batch = mnist.train.next_batch(50)
        if i % 100 == 0:
            run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
            run_metadata = tf.RunMetadata()
            summary, train_accuracy = sess.run([merged, accuracy], feed_dict={
              x: batch[0], y_: batch[1], keep_prob: 1.0}, options=run_options, run_metadata=run_metadata)
            print('step %d, training accuracy %g' % (i, train_accuracy))
            my_writer.add_run_metadata(run_metadata, 'step%d' % i)
            my_writer.add_summary(summary, i)
            
        else:    
            summary, _ = sess.run([merged, train_step], 
                                  feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
            my_writer.add_summary(summary, i)

    print('test accuracy %g' % accuracy.eval(feed_dict={
      x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.14
step 100, training accuracy 0.84
step 200, training accuracy 0.84
step 300, training accuracy 0.96
step 400, training accuracy 0.94
step 500, training accuracy 1
step 600, training accuracy 0.94
step 700, training accuracy 0.96
step 800, training accuracy 0.94
step 900, training accuracy 0.98
step 1000, training accuracy 0.94
step 1100, training accuracy 0.96
step 1200, training accuracy 0.96
step 1300, training accuracy 0.98
step 1400, training accuracy 0.96
step 1500, training accuracy 0.98
step 1600, training accuracy 1
step 1700, training accuracy 1
step 1800, training accuracy 0.94
step 1900, training accuracy 0.96
test accuracy 0.9777
